In [51]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [52]:
df = pd.read_csv('../data/raw/telco.csv')

In [53]:
df.sample(10)

,Customer ID,Gender,Age,Under 30,Senior Citizen,Married,Dependents,Number of Dependents,Country,State,...,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Satisfaction Score,Customer Status,Churn Label,Churn Score,CLTV,Churn Category,Churn Reason
4569,6429-SHBCB,Male,54,No,No,No,No,0,United States,California,...,0,261.44,1655.99,4,Stayed,No,77,3349,NaN,NaN
3867,6661-EIPZC,Female,38,No,No,Yes,Yes,1,United States,California,...,0,0.00,1423.05,4,Stayed,No,36,3194,NaN,NaN
5802,5707-ZMDJP,Male,61,No,No,Yes,Yes,2,United States,California,...,0,1679.57,5409.17,4,Stayed,No,27,4590,NaN,NaN
3032,3137-NYQQI,Male,32,No,No,Yes,No,0,United States,California,...,0,770.95,1946.55,4,Stayed,No,70,3546,NaN,NaN
3115,1051-GEJLJ,Female,19,Yes,No,No,No,0,United States,California,...,0,54.34,81.89,5,Joined,No,74,4836,NaN,NaN
3587,9469-WEJBT,Male,56,No,No,Yes,No,0,United States,California,...,150,258.63,3827.93,3,Stayed,No,76,3347,NaN,NaN
3910,2023-VQFDL,Male,42,No,No,No,No,0,United States,California,...,0,596.88,945.68,5,Stayed,No,39,5379,NaN,NaN
2220,5536-SLHPM,Female,38,No,No,Yes,No,0,United States,California,...,0,1067.55,5525.70,2,Churned,Yes,79,4398,Competitor,Competitor made better offer
239,4433-JCGCG,Male,80,No,Yes,Yes,No,0,United States,California,...,0,246.56,4926.61,1,Churned,Yes,88,2436,Competitor,Competitor made better offer
1829,7730-IUTDZ,Male,63,No,No,No,No,0,United States,California,...,0,1462.00,4660.60,1,Churned,Yes,68,4673,Dissatisfaction,Network reliability


In [54]:
df.shape


(7043, 50)

In [55]:
df.columns

Index(['Customer ID', 'Gender', 'Age', 'Under 30', 'Senior Citizen', 'Married',
       'Dependents', 'Number of Dependents', 'Country', 'State', 'City',
       'Zip Code', 'Latitude', 'Longitude', 'Population', 'Quarter',
       'Referred a Friend', 'Number of Referrals', 'Tenure in Months', 'Offer',
       'Phone Service', 'Avg Monthly Long Distance Charges', 'Multiple Lines',
       'Internet Service', 'Internet Type', 'Avg Monthly GB Download',
       'Online Security', 'Online Backup', 'Device Protection Plan',
       'Premium Tech Support', 'Streaming TV', 'Streaming Movies',
       'Streaming Music', 'Unlimited Data', 'Contract', 'Paperless Billing',
       'Payment Method', 'Monthly Charge', 'Total Charges', 'Total Refunds',
       'Total Extra Data Charges', 'Total Long Distance Charges',
       'Total Revenue', 'Satisfaction Score', 'Customer Status', 'Churn Label',
       'Churn Score', 'CLTV', 'Churn Category', 'Churn Reason'],
      dtype='object')

In [56]:
df.drop(columns=["Customer ID", "Latitude", "Longitude","Zip Code","Churn Reason","Churn Category","CLTV","Churn Score","Customer Status"], inplace=True)

In [57]:
df.isnull().sum() * 100 / df.shape[0]

Gender                                0.000000
Age                                   0.000000
Under 30                              0.000000
Senior Citizen                        0.000000
Married                               0.000000
Dependents                            0.000000
Number of Dependents                  0.000000
Country                               0.000000
State                                 0.000000
City                                  0.000000
Population                            0.000000
Quarter                               0.000000
Referred a Friend                     0.000000
Number of Referrals                   0.000000
Tenure in Months                      0.000000
Offer                                55.047565
Phone Service                         0.000000
Avg Monthly Long Distance Charges     0.000000
Multiple Lines                        0.000000
Internet Service                      0.000000
Internet Type                        21.666903
Avg Monthly G

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 41 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Gender                             7043 non-null   object 
 1   Age                                7043 non-null   int64  
 2   Under 30                           7043 non-null   object 
 3   Senior Citizen                     7043 non-null   object 
 4   Married                            7043 non-null   object 
 5   Dependents                         7043 non-null   object 
 6   Number of Dependents               7043 non-null   int64  
 7   Country                            7043 non-null   object 
 8   State                              7043 non-null   object 
 9   City                               7043 non-null   object 
 10  Population                         7043 non-null   int64  
 11  Quarter                            7043 non-null   objec

In [59]:
df['Internet Type'] = df['Internet Type'].fillna("No Internet Service")

In [60]:
df['Offer'] = df['Offer'].fillna("No Offer")

Dropping under 30 and Senior Citizen because we can directly indetify with age column

In [61]:
df.drop(columns=["Senior Citizen", "Under 30"], inplace=True)

Feature Reduction: Add-On Services

Original 8 columns: Online Security, Online Backup, Device Protection Plan, Premium Tech Support, Streaming TV, Streaming Movies, Streaming Music, Unlimited Data.

Reduced to 3 columns:

Security_Backup_Protection → counts security/backup/protection add-ons.

Streaming_Services → counts streaming services.

Unlimited_Data → binary (Yes → 1, No → 0).

In [62]:
df['Security_Backup_Protection'] = df[['Online Security', 'Online Backup', 
                                       'Device Protection Plan', 'Premium Tech Support']].apply(lambda x: x.eq('Yes').sum(), axis=1)

df['Streaming_Services'] = df[['Streaming TV', 'Streaming Movies', 'Streaming Music']].apply(lambda x: x.eq('Yes').sum(), axis=1)


df['Unlimited_Data'] = df['Unlimited Data'].replace({'Yes':1, 'No':0})

# Drop the original 8 columns
df.drop(columns=['Online Security', 'Online Backup', 'Device Protection Plan', 
                 'Premium Tech Support', 'Streaming TV', 'Streaming Movies', 
                 'Streaming Music', 'Unlimited Data'], inplace=True)


df.head()


C:\Users\Akbarhusain\AppData\Local\Temp\ipykernel_15436\137055147.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Unlimited_Data'] = df['Unlimited Data'].replace({'Yes':1, 'No':0})


,Gender,Age,Married,Dependents,Number of Dependents,Country,State,City,Population,Quarter,...,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Satisfaction Score,Churn Label,Security_Backup_Protection,Streaming_Services,Unlimited_Data
0,Male,78,No,No,0,United States,California,Los Angeles,68701,Q3,...,39.65,0.00,20,0.00,59.65,3,Yes,1,1,0
1,Female,74,Yes,Yes,1,United States,California,Los Angeles,55668,Q3,...,633.30,0.00,0,390.80,1024.10,3,Yes,1,0,1
2,Male,71,No,Yes,3,United States,California,Los Angeles,47534,Q3,...,1752.55,45.61,0,203.94,1910.88,2,Yes,0,3,1
3,Female,78,Yes,Yes,1,United States,California,Inglewood,27778,Q3,...,2514.50,13.43,0,494.00,2995.07,2,Yes,2,2,1
4,Female,80,Yes,Yes,1,United States,California,Whittier,26265,Q3,...,2868.15,0.00,0,234.21,3102.36,2,Yes,0,0,1


In [63]:
df.drop(columns=["Country", "State", "Population","Dependents","Total Long Distance Charges"], inplace=True)

In [64]:
df.shape

(7043, 29)

In [65]:
df.columns

Index(['Gender', 'Age', 'Married', 'Number of Dependents', 'City', 'Quarter',
       'Referred a Friend', 'Number of Referrals', 'Tenure in Months', 'Offer',
       'Phone Service', 'Avg Monthly Long Distance Charges', 'Multiple Lines',
       'Internet Service', 'Internet Type', 'Avg Monthly GB Download',
       'Contract', 'Paperless Billing', 'Payment Method', 'Monthly Charge',
       'Total Charges', 'Total Refunds', 'Total Extra Data Charges',
       'Total Revenue', 'Satisfaction Score', 'Churn Label',
       'Security_Backup_Protection', 'Streaming_Services', 'Unlimited_Data'],
      dtype='object')

In [66]:
for c in df.columns:
    print(c)
    print(df[c].unique())
    print("-" * 20)

Gender
['Male' 'Female']
--------------------
Age
[78 74 71 80 72 76 66 70 77 65 67 68 69 79 75 73 37 19 31 23 38 21 29 61
 27 20 56 51 48 32 34 41 30 26 62 64 45 53 63 42 24 54 39 43 50 22 40 47
 60 52 55 59 49 58 25 28 33 44 57 46 36 35]
--------------------
Married
['No' 'Yes']
--------------------
Number of Dependents
[0 1 3 2 5 4 6 7 8 9]
--------------------
City
['Los Angeles' 'Inglewood' 'Whittier' ... 'Topaz' 'Jacumba' 'Holtville']
--------------------
Quarter
['Q3']
--------------------
Referred a Friend
['No' 'Yes']
--------------------
Number of Referrals
[ 0  1  6  2  4  3  7  5  8  9 10 11]
--------------------
Tenure in Months
[ 1  8 18 25 37 27 58 15  7 11  3 13 16 24  4 32 54  2 21 55  5 20 65 49
 43 56 52 33 30 46  9 10 35 44 62 14 12 17 39 68 34 22 19 23 42 41 57 38
 29 66 69 40 31 59 36 45 53 70 47 50 51 72 26  6 28 48 61 71 60 64 67 63]
--------------------
Offer
['No Offer' 'Offer E' 'Offer D' 'Offer C' 'Offer B' 'Offer A']
--------------------
Phone Service
['No'